In [11]:
import datasets
import utils
import gmm_impute
import gmm_dae
import pickle_utils as pu
import numpy as np
import pandas as pd
from sklearn.mixture import BayesianGaussianMixture

import imp
imp.reload(utils)
imp.reload(datasets)
imp.reload(gmm_impute)
imp.reload(gmm_dae)

<module 'gmm_dae' from '/home/adria/MIMIC/ts-baselines/timeless-imputation/gmm_dae.py'>

In [9]:
dsets = datasets.datasets()
baseline = datasets.benchmark({'MICE': datasets.memoize(utils.impute_mice),
                               'MissForest': datasets.memoize(utils.impute_missforest)}, dsets)

/home/adria/MIMIC/ts-baselines/timeless-imputation/utils.py:26: RuntimeWarning: invalid value encountered in subtract
  sq_diff = (original_df-multiple_imputed_df)**2


In [5]:
BGMMs = {}
for name, df in dsets.items():
    m = BayesianGaussianMixture(n_components=100, covariance_type='full', max_iter=10000)
    m.fit(df.values)
    BGMMs[len(df)] = gmm_impute.prune_infinite_gmm(df.values, m)

In [6]:
m = BGMMs[len(dsets["BostonHousing"])]
from gmm_impute import mask_matrix
mask = np.zeros([m['means'].shape[1]], dtype=np.bool)
mask[[1,2,5,7,9,10]] = True
print("mask:", mask)
inp = dsets["BostonHousing"].values[0]

def compute_classically():
    K_12 = mask_matrix(m['covariances'],mask,~mask)
    K_22__1 = np.linalg.inv(mask_matrix(m['covariances'],~mask,~mask))
    K_1222 = K_12 @ K_22__1
    K_21 = mask_matrix(m['covariances'],~mask,mask)
    K_11 = mask_matrix(m['covariances'],mask,mask)

    diff = np.expand_dims(inp[~mask] - m['means'][:,~mask], axis=2)
    return (m['means'][:,mask] + np.squeeze(K_1222 @ diff, axis=2)), (K_11 - K_1222 @ K_21)

def compute_newly():
    K12 = m['covariances']*(mask[:,None])*(~mask)

mask: [False  True  True False False  True False  True False  True  True False]


In [12]:
%%time
def gmm_impute_f(_path, dataset):
    _dataset = dataset.values
    ndarray = gmm_impute._gmm_impute(BGMMs[len(dataset)], _dataset)
    return list(pd.DataFrame(a, index=dataset.index, columns=dataset.columns) for a in ndarray)
benchmark_gmm = datasets.benchmark({'GMM': gmm_impute_f}, dsets)

CPU times: user 57.5 s, sys: 2min 6s, total: 3min 3s
Wall time: 23.2 s


In [ ]:
%%time
def ff(p, d):
    return gmm_dae.gmm_dae_impute(p, d, BGMMs[len(d)])
gmm_dae = datasets.benchmark({'GMM_dae': ff}, dsets)

In [ ]:
print("RECALL: the GMMs are learned from the full data.")
pd.concat([baseline, benchmark_gmm, gmm_dae], axis=1)